In [1]:
import tabula
import pdfplumber
import pandas as pd
import numpy as np

In [2]:
pdf_path = "missori1.pdf"
# pdf_path = "pdf//5.pdf"

In [3]:
def extract_text_coordinates(pdf_path):
    text_list = []
    with pdfplumber.open(pdf_path) as pdf:
        # Iterate through each page of the PDF
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]
            page_number = page_number + 1
            # Extract the text content and its coordinates
            for element in page.extract_words(x_tolerance=1, y_tolerance=1, keep_blank_chars=True):
                text_dict = {}
                text = element["text"]
                x, y = element["x0"], element["top"]
                
                text_cord = f"Page {page_number + 1}: Text: {text} Coordinates: X={x}, Y={y}"
                text_dict.update({'page':page_number, 'text':text,'cordx':x,'cordy':y})
                text_list.append(text_dict)
    return text_list

text_list = extract_text_coordinates(pdf_path)

In [4]:
text_list

[{'page': 1,
  'text': '1 of 3',
  'cordx': 438.70334,
  'cordy': 23.429320000000075},
 {'page': 1,
  'text': 'Delta Dental of Missouri',
  'cordx': 54.14185,
  'cordy': 54.144540000000006},
 {'page': 1,
  'text': 'c/o Customer Service',
  'cordx': 54.14185,
  'cordy': 62.144540000000006},
 {'page': 1, 'text': 'P.O. Box 8690', 'cordx': 54.14185, 'cordy': 70.14454},
 {'page': 1,
  'text': 'St Louis MO 63126',
  'cordx': 54.14185,
  'cordy': 78.14454},
 {'page': 1,
  'text': 'Explanation of Benefits',
  'cordx': 347.57030000000003,
  'cordy': 88.38616000000002},
 {'page': 1,
  'text': 'RETAIN FOR TAX PURPOSES',
  'cordx': 368.1293,
  'cordy': 111.24338000000012},
 {'page': 1,
  'text': 'THIS IS NOT A BILL',
  'cordx': 392.15930000000003,
  'cordy': 122.58194000000003},
 {'page': 1,
  'text': '                    JECB',
  'cordx': 54.14185,
  'cordy': 151.97546999999997},
 {'page': 1,
  'text': '22,745',
  'cordx': 235.56049000000002,
  'cordy': 151.97451999999998},
 {'page': 1,
  'text':

[{'page': 1,
  'text': '1 of 3',
  'cordx': 438.70334,
  'cordy': 23.429320000000075},
 {'page': 1,
  'text': 'Delta Dental of Missouri',
  'cordx': 54.14185,
  'cordy': 54.144540000000006},
 {'page': 1,
  'text': 'c/o Customer Service',
  'cordx': 54.14185,
  'cordy': 62.144540000000006},
 {'page': 1, 'text': 'P.O. Box 8690', 'cordx': 54.14185, 'cordy': 70.14454},
 {'page': 1,
  'text': 'St Louis MO 63126',
  'cordx': 54.14185,
  'cordy': 78.14454},
 {'page': 1,
  'text': 'Explanation of Benefits',
  'cordx': 347.57030000000003,
  'cordy': 88.38616000000002},
 {'page': 1,
  'text': 'RETAIN FOR TAX PURPOSES',
  'cordx': 368.1293,
  'cordy': 111.24338000000012},
 {'page': 1,
  'text': 'THIS IS NOT A BILL',
  'cordx': 392.15930000000003,
  'cordy': 122.58194000000003},
 {'page': 1,
  'text': '                    JECB',
  'cordx': 54.14185,
  'cordy': 151.97546999999997},
 {'page': 1,
  'text': '22,745',
  'cordx': 235.56049000000002,
  'cordy': 151.97451999999998},
 {'page': 1,
  'text':

In [5]:
dfs = []
x2 = 0
y2 = 0
for text_d in text_list:
    x = text_d['cordx']
    y = text_d['cordy']
    if x2<x:
        x2 = text_d['cordx']
    if y2<y:
        y2 = text_d['cordy']    
x2 = x2+20
# y2 = y2+20
print("YYYY", y2)
for i in range(len(text_list)):
    x1 = y1 = ''
    if 'Tooth' in text_list[i]['text']:
        print("GRISER, ARIA", text_list[i]['text'])
        x1 = text_list[i]['cordx']
        y1 = text_list[i]['cordy']
        x1 = x1 - 120
        y1 = y1 + 5
#         print("X1,Y1", x1,y1)
        page_number = text_list[i]['page']
        for j in range(i,len(text_list)):
            if page_number == text_list[j]['page']:
                if 'Claim' in text_list[j]['text']:
                    print("Bene", text_list[j]['text'])
                    print("PAG", text_list[j]['page'] )
                    page = int(text_list[j]['page'])
                    y2 = text_list[j]['cordy']
                    print(f"x1 = {x1}, y1 = {y1}, x2 = {x2}, y2 = {y2}")
                    tabula_dfs = tabula.read_pdf(pdf_path, guess=False, pages=page , stream=True, encoding="utf-8",
                                     area=(y1, x1, y2, x2), multiple_tables=True)
                    df_filled = tabula_dfs[0].fillna('')
                    dfs.append(df_filled[1:])
                    break
            
            if page_number == text_list[j]['page']:
                if '10 Hudson Yards' in text_list[j]['text']:
                    print("10 Hudson", text_list[j]['text'])
                    print("PAG", text_list[j]['page'] )
                    page = int(text_list[j]['page'])
                    y2 = text_list[j]['cordy']
                    print(f"x1 = {x1}, y1 = {y1}, x2 = {x2}, y2 = {y2}")
                    tabula_dfs = tabula.read_pdf(pdf_path, guess=False, pages=page , stream=True, encoding="utf-8",
                                     area=(y1, x1, y2, x2), multiple_tables=True)
                    df_filled = tabula_dfs[0].fillna('')
                    dfs.append(df_filled[1:])
                    break
                
            
            if page_number == text_list[j]['page']:
#                 if 'BENEFIT SUMMARY' not in text_list[j]['text'] and '10 Hudson Yards' not in text_list[j]['text']:
                if 'VNE' in text_list[j]['text']:
                    y2 = 780
                    page = int(text_list[j]['page'])
                    print(f"x1 = {x1}, y1 = {y1}, x2 = {x2}, y2 = {y2}")
                    tabula_dfs = tabula.read_pdf(pdf_path, guess=False, pages=page , stream=True, encoding="utf-8",
                                     area=(y1, x1, y2, x2), multiple_tables=True)
                    df_filled = tabula_dfs[0].fillna('')
                    dfs.append(df_filled[1:])
                    break
   

YYYY 808.80914
GRISER, ARIA Tooth
Bene Claim No:
PAG 1
x1 = 25.832510000000013, y1 = 287.05575999999996, x2 = 891.1765599999999, y2 = 497.77322
YYYY 808.80914
GRISER, ARIA Tooth
Bene Claim No:
PAG 1
x1 = 25.832510000000013, y1 = 287.05575999999996, x2 = 891.1765599999999, y2 = 497.77322
GRISER, ARIA Tooth
Bene Claim No:
PAG 1
x1 = 58.43093999999999, y1 = 287.05575999999996, x2 = 891.1765599999999, y2 = 497.77322
GRISER, ARIA Tooth
Bene Claim No:
PAG 1
x1 = 58.43093999999999, y1 = 287.05575999999996, x2 = 891.1765599999999, y2 = 497.77322
GRISER, ARIA Tooth
GRISER, ARIA Tooth
GRISER, ARIA Tooth
Bene Claim No:
PAG 2
x1 = 25.832510000000013, y1 = 140.05576000000008, x2 = 891.1765599999999, y2 = 291.24558
GRISER, ARIA Tooth
GRISER, ARIA Tooth
GRISER, ARIA Tooth
Bene Claim No:
PAG 2
x1 = 25.832510000000013, y1 = 140.05576000000008, x2 = 891.1765599999999, y2 = 291.24558
GRISER, ARIA Tooth
Bene Claim No:
PAG 2
x1 = 58.43093999999999, y1 = 140.05576000000008, x2 = 891.1765599999999, y2 = 291.

In [6]:
print(len(dfs))
dfs[0]

6


,Date,Code Code,Number Surface,Unnamed: 0,Unnamed: 1,Amount Amount,Unnamed: 2,Amount,Deductible,Unnamed: 3,%,Pays,Pays.1,Notice*
1,/22/23,D0230 D0230,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051
2,/22/23,D0220 D0220,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051 1075
3,/22/23,D0120 D0120,,,,$85.00 $66.00,,$66.00,$0.00,$0.00,100%,$0.00,$66.00,40389
4,/22/23,D0274 D0210,,,,$205.00 $88.00,,$88.00,$0.00,$0.00,0%,$88.00,$0.00,1051 1075
5,/22/23,D1206 D1206,,,,$60.00 $31.00,,$31.00,$0.00,$0.00,100%,$0.00,$31.00,40389
6,/22/23,D0230 D0230,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051
7,,,Claim Totals:,,,$445.00 $230.00,,$230.00,$0.00,$0.00,,$88.00,$142.00,
8,,Total Patient Responsibility:,,,,$88.00,,,,,,,,


6


,Date,Code Code,Number Surface,Unnamed: 0,Unnamed: 1,Amount Amount,Unnamed: 2,Amount,Deductible,Unnamed: 3,%,Pays,Pays.1,Notice*
1,/22/23,D0230 D0230,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051
2,/22/23,D0220 D0220,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051 1075
3,/22/23,D0120 D0120,,,,$85.00 $66.00,,$66.00,$0.00,$0.00,100%,$0.00,$66.00,40389
4,/22/23,D0274 D0210,,,,$205.00 $88.00,,$88.00,$0.00,$0.00,0%,$88.00,$0.00,1051 1075
5,/22/23,D1206 D1206,,,,$60.00 $31.00,,$31.00,$0.00,$0.00,100%,$0.00,$31.00,40389
6,/22/23,D0230 D0230,,,,$0.00 $0.00,,$0.00,$0.00,$0.00,0%,$0.00,$0.00,1051
7,,,Claim Totals:,,,$445.00 $230.00,,$230.00,$0.00,$0.00,,$88.00,$142.00,
8,,Total Patient Responsibility:,,,,$88.00,,,,,,,,


In [7]:
search_string = 'TOTALS'
final = []
stack_list = []
for d in dfs:
    last_row = d.iloc[-1]
    col = last_row[4]
    
    print(col)
    print(type(col))
    if 'TOTALS' in col:
        print("In the if")
        if stack_list:
            new_df = pd.concat(stack_list + [d] , axis=0)
            final.append(new_df)
            stack_list = []
        else:
            final.append(d)    
    else:
        stack_list.append(d)

        


<class 'str'>
$88.00
<class 'str'>

<class 'str'>
$0.00
<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>
$88.00
<class 'str'>

<class 'str'>
$0.00
<class 'str'>

<class 'str'>

<class 'str'>


In [8]:
print(len(final))

0
0


In [9]:
tdfs =[]

for df in final:
    df = df[~df.apply(lambda row: row.astype(str).str.contains('TOTALS', case=False)).any(axis=1)]
    df.rename(columns={'Date of': 'Date'}, inplace=True)
    df.fillna('', inplace=True)
    df.drop('Line', axis=1, inplace=True)
    tdfs.append(df)
   

In [10]:
tdfs[0]

IndexError: list index out of range

IndexError: list index out of range

In [ ]:
new_dict = []
for td in tdfs:
    tab = td.to_dict(orient='records')
    for i in range(len(tab)):
        new_dict.append(tab[i])
    
#     new_dict.append(td.to_dict(orient='records'))

In [ ]:
print(len(new_dict))

In [ ]:
tab_dict = []

change_keys = [("Alt","AltCode"),("Tooth", "ToothNo"),("Date", "ServiceDate"), ("Submitted.1", "SubmittedCharges"),
               ("Benefit", "PayableAmount"), ("Considered", "ActualAllowed"),
               ("Deductible", "ContractualObligations"),("Covered", "CoveredCharge"),("Coverage", "CoveragePercent")]
for d in new_dict:
    for k in change_keys:
        old_key = k[0]
        new_key = k[1]
        value = d[old_key]
        d[new_key] = value
        
        
    for k in change_keys:
        del d[k[0]]
    print("D",d)
    tab_dict.append(d)
    

In [ ]:
tab_dict

In [ ]:
# dfs = []
# x2 = 0
# for text_d in text_list:
#     x = text_d['cordx']
#     if x2<x:
#         x2 = text_d['cordx']
# x2 = x2+20      
# for i in range(len(text_list)):
#     x1 = y1 = y2 = ''
#     if 'Line' in text_list[i]['text']:
#         print("Line", text_list[i]['text'])
#         x1 = text_list[i]['cordx']
#         y1 = text_list[i]['cordy']
#         page_number = text_list[i]['page'])
#         for j in range(i,len(text_list)):
#             if 'BENEFIT SUMMARY' in text_list[j]['text']:
#                 print("Bene", text_list[j]['text'])
#                 page = int(text_list[j]['page'])
#                 y2 = text_list[j]['cordy']
#                 print(f"x1 = {x1}, y1 = {y1}, x2 = {x2}, y2 = {y2}")
#                 tabula_dfs = tabula.read_pdf(pdf_path, guess=False, pages=page , stream=True, encoding="utf-8",
#                                  area=(y1, x1, y2, x2), multiple_tables=True)
#                 dfs.append(tabula_dfs[0])
                
#                 break

In [11]:
tabula_dfs = tabula.read_pdf(pdf_path, guess=False, pages=1 , stream=True, encoding="utf-8",
                                     area=(525, 18, 647, 598), multiple_tables=True)

In [12]:
tabula_dfs[0]

,Service,Submitted,Processed,Tooth Tooth,Unnamed: 0,Submitted.1,Accepted,Allowed,Applied,Co-Pay,Co-Ins,Patient,Plan,Adjustment
0,Date,Code,Code,Number Surface,NaN,Amount,Amount,Amount,Deductible,NaN,%,Pays,Pays,Notice*
1,6/22/23,D0120,D0120,NaN,NaN,$85.00,$66.00,$66.00,$0.00,$0.00,100%,$0.00,$66.00,40389
2,6/22/23,D0272,D0272,NaN,NaN,$65.00,$27.00,$27.00,$0.00,$0.00,100%,$0.00,$27.00,40389
3,6/22/23,D1120,D1120,NaN,NaN,$95.00,$45.00,$45.00,$0.00,$0.00,100%,$0.00,$45.00,40389
4,6/22/23,D1206,D1206,NaN,NaN,$60.00,$31.00,$31.00,$0.00,$0.00,100%,$0.00,$31.00,40389
5,NaN,NaN,NaN,Claim Totals:,NaN,$305.00,$169.00,$169.00,$0.00,$0.00,NaN,$0.00,$169.00,NaN


,Service,Submitted,Processed,Tooth Tooth,Unnamed: 0,Submitted.1,Accepted,Allowed,Applied,Co-Pay,Co-Ins,Patient,Plan,Adjustment
0,Date,Code,Code,Number Surface,NaN,Amount,Amount,Amount,Deductible,NaN,%,Pays,Pays,Notice*
1,6/22/23,D0120,D0120,NaN,NaN,$85.00,$66.00,$66.00,$0.00,$0.00,100%,$0.00,$66.00,40389
2,6/22/23,D0272,D0272,NaN,NaN,$65.00,$27.00,$27.00,$0.00,$0.00,100%,$0.00,$27.00,40389
3,6/22/23,D1120,D1120,NaN,NaN,$95.00,$45.00,$45.00,$0.00,$0.00,100%,$0.00,$45.00,40389
4,6/22/23,D1206,D1206,NaN,NaN,$60.00,$31.00,$31.00,$0.00,$0.00,100%,$0.00,$31.00,40389
5,NaN,NaN,NaN,Claim Totals:,NaN,$305.00,$169.00,$169.00,$0.00,$0.00,NaN,$0.00,$169.00,NaN
